<a href="https://colab.research.google.com/github/zyz314/100-Days-Of-ML-Code/blob/master/project8_Manual_Strategy_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 导入必要的库
import pandas as pd  # 用于数据处理
import datetime as dt  # 用于处理日期时间
import matplotlib.pyplot as plt  # 用于绘图

# 定义手动策略类
class ManualStrategy:
    def __init__(self, verbose=False, impact=0.0, commission=0.0):
        self.verbose = verbose  # 是否打印调试信息
        self.impact = impact  # 交易冲击
        self.commission = commission  # 佣金

    def author(self):
        return 'your_gt_username'  # 返回GT用户名

    def study_group(self):
        return 'your_study_group'  # 返回学习小组成员的GT用户名，多个用户名用逗号分隔

    def add_evidence(self, symbol='IBM', sd=dt.datetime(2008, 1, 1), ed=dt.datetime(2009, 1, 1), sv=100000):
        pass  # add_evidence方法用于训练策略，这里假设不做任何操作，仅作为占位

    def testPolicy(self, symbol='JPM', sd=dt.datetime(2009, 1, 1), ed=dt.datetime(2010, 1, 1), sv=100000):
        dates = pd.date_range(sd, ed)  # 生成日期范围
        prices_all = self.get_data([symbol], dates)  # 获取所有股票数据
        prices = prices_all[symbol]  # 获取目标股票的数据

        sma = self.compute_sma(prices, window=20)  # 计算简单移动平均线（SMA）
        bb_upper, bb_lower, bb_percent = self.compute_bollinger_bands(prices, window=20)  # 计算布林带
        momentum = self.compute_momentum(prices, window=10)  # 计算动量

        trades = pd.DataFrame(index=prices.index, columns=[symbol])  # 以日期为索引，股票代码为列名创建DataFrame
        trades.iloc[:, :] = 0  # 初始化为0（无持仓）

        position = 0  # 初始化持仓状态，0: 无持仓，1: 多头，-1: 空头

        for i in range(1, len(prices)):  # 遍历每个交易日的价格数据
            sma_today = sma.iloc[i]  # 获取当天的SMA值
            bb_today = bb_percent.iloc[i]  # 获取当天的布林带百分比值
            momentum_today = momentum.iloc[i]  # 获取当天的动量值

            if (prices.iloc[i] < sma_today and bb_today < 0 and momentum_today < 0) and position == 0:  # 判断买入信号
                trades.iloc[i, trades.columns.get_loc(symbol)] = 1000  # 买入1000股
                position = 1  # 更新持仓状态为多头

            elif (prices.iloc[i] > sma_today and bb_today > 1 and momentum_today > 0) and position == 1:  # 判断卖出信号
                trades.iloc[i, trades.columns.get_loc(symbol)] = -2000  # 卖出2000股（平多头并做空1000股）
                position = -1  # 更新持仓状态为空头

            elif (prices.iloc[i] < sma_today and bb_today < 0 and momentum_today < 0) and position == -1:  # 判断平空头信号
                trades.iloc[i, trades.columns.get_loc(symbol)] = 1000  # 买入1000股（平空头）
                position = 0  # 更新持仓状态为无持仓

            else:  # 无交易信号，保持现有状态
                trades.iloc[i, trades.columns.get_loc(symbol)] = 0  # 无交易

        return trades  # 返回交易信号DataFrame

    def compute_sma(self, prices, window=20):
        """计算简单移动平均线（SMA）"""
        sma = prices.rolling(window=window).mean()  # 计算移动平均值
        return sma

    def compute_bollinger_bands(self, prices, window=20):
        """计算布林带"""
        sma = self.compute_sma(prices, window)  # 计算SMA
        rolling_std = prices.rolling(window=window).std()  # 计算滚动标准差
        upper_band = sma + (2 * rolling_std)  # 计算上布林带
        lower_band = sma - (2 * rolling_std)  # 计算下布林带
        bb_percent = (prices - lower_band) / (upper_band - lower_band)  # 计算布林带百分比
        return upper_band, lower_band, bb_percent

    def compute_momentum(self, prices, window=10):
        """计算动量"""
        momentum = prices / prices.shift(window) - 1  # 计算动量
        return momentum

    def get_data(self, symbols, dates):
        """读取给定符号的股票数据（调整后的收盘价）"""
        df = pd.DataFrame(index=dates)  # 以日期为索引创建DataFrame
        for symbol in symbols:
            df_temp = pd.read_csv(f"data/{symbol}.csv", index_col='Date', parse_dates=True,
                                  usecols=['Date', 'Adj Close'], na_values=['nan'])  # 读取CSV文件中的股票数据
            df_temp = df_temp.rename(columns={'Adj Close': symbol})  # 重命名列名为股票代码
            df = df.join(df_temp)  # 将数据合并到主DataFrame中
            if symbol == 'SPY':
                df = df.dropna(subset=["SPY"])  # 删除SPY数据中的NaN值
        return df  # 返回股票数据DataFrame

    def plot_results(self, portvals, df_trades, symbol='JPM'):
        """绘制结果图表"""
        plt.figure(figsize=(10, 6))  # 设置图表大小
        plt.plot(portvals, label='Manual Strategy', color='red')  # 绘制手动策略的投资组合价值曲线
        plt.axvline(x=df_trades[df_trades[symbol] == 1000].index, color='blue', linestyle='--', label='Buy Signal')  # 绘制买入信号线
        plt.axvline(x=df_trades[df_trades[symbol] == -1000].index, color='black', linestyle='--', label='Sell Signal')  # 绘制卖出信号线
        plt.legend(loc='best')  # 显示图例
        plt.xlabel('Date')  # 设置X轴标签
        plt.ylabel('Normalized Portfolio Value')  # 设置Y轴标签
        plt.title('Manual Strategy vs Benchmark')  # 设置图表标题
        plt.show()  # 显示图表

# 示例用法
if __name__ == "__main__":
    ms = ManualStrategy(verbose=True, impact=0.005, commission=9.95)  # 创建手动策略实例，带有调试信息、交易冲击和佣金
    ms.add_evidence(symbol='JPM', sd=dt.datetime(2008, 1, 1), ed=dt.datetime(2009, 1, 1), sv=100000)  # 训练策略（这里实际上不做任何操作）
    df_trades = ms.testPolicy(symbol='JPM', sd=dt.datetime(2009, 1, 1), ed=dt.datetime(2010, 1, 1), sv=100000)  # 生成交易数据
    portvals = compute_portvals(df_trades, start_val=100000, commission=9.95, impact=0.005)  # 计算投资组合价值

    portvals = portvals / portvals.iloc[0]  # 归一化投资组合价值到1.0

    ms.plot_results(portvals, df_trades, symbol='JPM')  # 调用绘图函数绘制结果图表
